In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel, PeftConfig
import torch

import json
import re
from nltk.stem import PorterStemmer
from rouge import Rouge
from tqdm import tqdm

/home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64${LD_LIBRARY_PATH'), PosixPath('}'), PosixPath('+')}
  warn(msg)
/home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/cuda/lib64${LD_LIBRARY_PATH:+:} did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to b

In [2]:
def filter_text(text):
    # Оставляем только буквы, цифры, знаки препинания и пробелы
    filtered_text = re.sub(r'[^а-яА-Яa-zA-Z0-9.,!?;:\-\'" \n<>/]', '', text)

    # Заменяем повторяющиеся символы
    filtered_text = re.sub(r'([.,!?;:\-\'"])\1+', r'\1', filtered_text)
    
    return filtered_text

In [3]:
points = ["название", "специализация", "требования", "условия", 'описание', 'обязанности']
points = ["\"" + p + "\"" for p in points]


example_text = \
"""""Срочно требуются бетонщики- монолитчики на работу вахтовым методом!   Tpeбования : Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки. Умение читать чертежи является преимуществом. У вас будет: - Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет. - Питание – за наш счет - Компенсация проезда до места вахты.  Рассматриваем также готовые бригады! Честную оплату труда - гарантируем!\""""

        
short_example_ans = \
""""название": "Бетонщик"
"специализация": "Бетонщики (Вахта,питание, проживание,проезд)"
"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки."
"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет."
"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"
"обязанности": "Мешать бетон стройке\""""


example_ans = \
""""название": "Бетонщик"
"специализация": "Бетонщики (Вахта,питание, проживание,проезд)"
"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки.Умение читать чертежи является преимуществом. Рассматриваем также готовые бригады!"
"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет. - Питание – за наш счет - Компенсация проезда до места вахты. Честную оплату труда - гарантируем!"
"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"
"обязанности": """

In [4]:
query = \
"""
"Требуются бетонщики ВАХТОВЫМ методом работы в НОВОСИБИРСК!  
БЕСПЛАТНОЕ ПРОЖИВАНИЕ (со всеми удобствами, без живностей)  3-Х РАЗОВОЕ ПИТАНИЕ!  
Ставка почасовая  🚉БЕСПЛАТНЫЙ ПРОЕЗД С ВАШЕГО ГОРОДА ДО ОБЪЕКТА И ОБРАТНО!!!  
✔️ Требования:  — Опыт работы от 2-х лет  ✔️ Обязанности:  — Заливка бетона  
✔️ Условия:  ПРЯМОЙ РАБОТОДАТЕЛЬ!  УДЕРЖАНИЙ НИКАКИХ НЕТ!!!  
— Оплата с первого дня; — Полностью официальное оформление по ТК; — 
БЕСПЛАТНОЕ проживание в квартире (условия хорошие, без живностей. 
Все для комфортного проживания предоставляется!); — Спецодежда выдается бесплатно; — 
Стабильная работа без задержек; — Работа вахтовым методом на выбор 60/30, 90/30. — 
Корпоративный транспорт от места проживания до работы и обратно;  
Продление вахты возможно на месте. 90% сотрудников продлевают вахту и остаются с нами.  
Присоединяйтесь к нашей дружной команде!  Звоните и пишите в удобное для вас время!  
Также рассмотрим кандидатов на такие должности, как: каменщик, монтажник ТТ, монтажник НТ"
"""


query2 = \
"""
Требования:Разряд 4;Опыт работы более 3 лет Обязанности:Ремонт металлорежущего и другого оборудования Условия:Труда нормальные;Рабочий день полный с 9-00 до 18-00;Обеденный перерыв с 13-00 до 14-00;Заработная плата от 30 тыс. рублей
"""

query3 = \
""""Требуется АРМАТУРЩИК на крупнейший завод по производству железобетонных конструкций 3-4 разряда!  Ставка за смену 3080 руб при графике 6/1 по 11 часов.  Предоставляем час обеда + перерывы 10-15 мин. Обязанности: Вязка арматуры для заливки конструкции железобетонных изделий.  Требования: Опыт работы от 1 года , наличие удостоверения (документы должны присутствовать + запись в трудовой книжке). Желание работать и зарабатывать Условия:  Работа вахтой от 45/60/90/120 смен; - Трудоустройство и заселение в общежитие в день обращения; - Бесплатное проживание в комфортном общежитии (постельное бельё предоставляем);  - Питание 1 раз в день предоставляется на заводе - В общежитии есть кухня и все необходимое для приготовления пищи; - Работа в закрытом теплом цехе - Удобный график работы ночь или день; -Вахта от 45 смен, зп на руки 138 600. Иногородним предоставляется проживание бесплатное в общежитии  Объект расположен по адресу : гор. Подпорожье Ленинградская область.  Рассматриваем кандидатов с опытом работы: арматурщик, строитель, разнорабочий. работа вахтой в Санкт-Петербурге, вакансия вахта, вахтовый метод, вахта прямой работодатель, работа вахтой без опыта, работа с проживанием прямой работодатель, работа вакансии, работа вахтой вакансии, разнорабочий, рабочий, сборщик, подсобный рабочий, работник завода,работа в москве и подмосковье вахтой, формовщик."""

query4 = \
"Крупной Федеральной Компании для работы ВАХТОВЫМ МЕТОДОМ на строительные участки требуются специалисты:  Обязанности:  · Работы на монолите (заливка бетона, укладка и вязка арматуры)  · Монтажные работы (МК/ЖБК)  · Общестроительные работы и вспомогательные работы  Требования:  · Понимание работы в бригаде  · Дисциплинированность, трудолюбие  · Опыт работы в строительстве будет преимуществом  · (готовы рассмотреть кандидатов и без опыта работы, всему научим, в перспективе карьерный рост до бригадира/мастера участка)  Условия:  · Продолжительность вахты 60/30, 90/30 (можно больше)  · Официальное трудоустройство по ТК РФ с первой рабочей смены  · ЗП строго в срок и без задержек (документальное подтверждение в трудовом договоре)  · Авансирование (15 и 30го числа каждого месяца по 15000 р.)  · Обеспечим сезонной спецодеждой и СИЗами без вычетов из ЗП  · Организованное трехраховое горячее питание за счет компании  · Организованные отправки до объектов (покупаем билеты на вахту/с вахты)  · Помощь в прохождении мед. осмотра (при необходимости)  · Возможность получить квалификационное удостоверение (обучение основным и смежным специальностям в нашем аккредитованном учебном центре)"

query5 = \
"Вас приветствует компания ООО «ПРОФКАДРЫ» -лидер кадрового обеспечения крупных  предприятий России.  На крупное предприятие в городе Тихвин требуется Маляр ,  для работы вахтовым методом.  Обязанности:  Работы по нанесению покрытий на металлоконструкции при изготовлении отливок, узлов, вагонов (лакокрасочным и материалами по металлу)  — Требования: — Опыт работы от 1-3 лет. — Наличие удостоверении по специальности и запись в трудовой книжке, подтверждающая квалификацию.  Условия:  — Официальное трудоустройство по ТК РФ. — Оплата отпуска и больничного. — Своевременная зарплата 2 р в месяц на карту любого банка. — Предоставление благоустроенного жилья. — Заселение в день приезда. — Спецодежда и медкомиссия на месте за счёт работодателя. — Компенсация дороги по билетам. — Оплата почасовая 290 р/ч. — График 6/1,по 11 часов, перерывом на обед. — Вахта 60/30,30/30.  Работа в стабильной, динамично развивающейся компании.  Возможность карьерного роста.  Проводим адаптацию и даём полную поддержку новым сотрудникам."

In [5]:
# Приведу пример вакансии и ответа:
# <example>
# Текст вакансии: 
# <text>
# {example_text}
# </text>
# Для такой вакансии хорошим ответом будет:
# <answer>
# {example_ans}
# </answer> 
# </example>

In [6]:

get_prompt = lambda q: f'''Задание: Дан текст вакансии, выдели из этого текста пункты {', '.join(points)} и напиши их в такой форме:
<format-example>
{short_example_ans}
</format-example>
Если какой-то пункт отсутствует, то оставь после него пустую строку. Пункты кроме {', '.join(points)} писать запрещено.
Повторять одинаковые пункты несколько раз запрещено.
Приведу пример вакансии и ответа:
<example>
Текст вакансии: 
<text>
{example_text}
</text>
Для такой вакансии хорошим ответом будет:
<answer>
{example_ans}
</answer> 
</example>
Теперь я дам тебе настоящий текст вакансии, выдели из него нужные пункты в заданном формате.
Текст вакансии:
<text>
{q}
</text>
Ответ:
'''
inputs = [filter_text(get_prompt(i)) for i in [query, query2, query3, query4, query5]]

# inputs = inputs * 4

In [7]:
print(inputs[1])

Задание: Дан текст вакансии, выдели из этого текста пункты "название", "специализация", "требования", "условия", "описание", "обязанности" и напиши их в такой форме:
<format-example>
"название": "Бетонщик"
"специализация": "Бетонщики Вахта,питание, проживание,проезд"
"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки."
"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет."
"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"
"обязанности": "Мешать бетон стройке"
</format-example>
Если какой-то пункт отсутствует, то оставь после него пустую строку. Пункты кроме "название", "специализация", "требования", "условия", "описание", "обязанности" писать запрещено.
Повторять одинаковые пункты несколько раз запрещено.
Приведу пример вакансии и ответа:
<example>
Текст вакансии: 
<text>
"Срочно требуются бетонщики- монолитчики на работу вахтовым методом!   Tpeбования : Опыт рабoты на монол

In [8]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# MODEL_NAME = "IlyaGusev/saiga_13b_lora"
MODEL_NAME = "IlyaGusev/llama_7b_ru_turbo_alpaca_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты - профессиональный HR. Далее ты будет работать с текстами вакансий и обрабатывать их. Строго следуй условиям заданий."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()

def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


In [9]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 33/33 [00:06<00:00,  4.82it/s]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        

In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)


# setattr(generation_config, "temperature", 0.2)
# setattr(generation_config, "top_k", 30)


print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 512,
  "no_repeat_ngram_size": 20,
  "num_beams": 3,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.95,
  "top_k": 30,
  "top_p": 0.85,
  "transformers_version": "4.30.2"
}



In [11]:
def parse_model_answer(text: str):
    rows = text.split('\n')
    res = {}
    for row in rows:
        for point in points:
            if row.startswith(point):
                key = point.replace("\"", "")
                res[key] = row.replace(point + ": ", "").replace("\"", "")
    return res

In [12]:
for inp in inputs[:2]:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(output)
    res = parse_model_answer(output)
    print(json.dumps(res, indent=2, ensure_ascii=False))
    print()
    print("==============================")
    print()

ox injections for wrinkles

Botox injections for wrinkles are a popular cosmetic procedure used to reduce the appearance of wrinkles and fine lines on the face. Botox is a brand name for botulinum toxin type A, a purified protein produced by the bacterium Clostridium botulinum. Botox injections for wrinkles work by temporarily paralyzing the muscles that cause wrinkles, resulting in a smoother, more youthful appearance.
{}


ox injections for wrinkles

Botox injections for wrinkles are one of the most popular cosmetic procedures in the world. Botox is a neurotoxin that temporarily paralyzes the muscles that cause wrinkles. It can be used to reduce the appearance of wrinkles on the forehead, between the eyebrows, and around the eyes.

Botox injections for wrinkles are safe and effective when administered by a qualified medical professional. The procedure involves injecting small amounts of Botox into the muscles that cause wrinkles. The Botox temporarily paralyzes the muscles, which red

In [14]:
data = tokenizer(["### Задание: Почему трава зеленая?\n### Ответ: "], return_tensors="pt")
data = {k: v.to(model.device) for k, v in data.items() if k in ("input_ids", "attention_mask")}
output_ids = model.generate(**data, generation_config=generation_config)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True))



### Задание: Почему трава зеленая?
### Ответ:  Трава зеленая - это результат фотосинтеза, процесса, который осуществляется зелеными растениями для получения энергии и питательных веществ из воздуха и воды. В процессе фотосинтеза зеленые растения используют солнечную энергию, чтобы превратить углекислый газ в органические вещества, такие как углеводы, аминокислоты и жирные кислоты. Эти вещества используются растениями для создания питательных веществ, которые необходимы для их роста и развития.


In [13]:
prompt

'<s>system\nТы - профессиональный HR. Далее ты будет работать с текстами вакансий и обрабатывать их. Строго следуй условиям заданий.</s>\n<s>user\nЗадание: Дан текст вакансии, выдели из этого текста пункты "название", "специализация", "требования", "условия", "описание", "обязанности" и напиши их в такой форме:\n<format-example>\n"название": "Бетонщик"\n"специализация": "Бетонщики Вахта,питание, проживание,проезд"\n"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки."\n"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет."\n"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"\n"обязанности": "Мешать бетон стройке"\n</format-example>\nЕсли какой-то пункт отсутствует, то оставь после него пустую строку. Пункты кроме "название", "специализация", "требования", "условия", "описание", "обязанности" писать запрещено.\nПовторять одинаковые пункты несколько раз запрещено.\nПриведу пример

In [14]:
samples = [
    {
        "text": "Вас приветствует компания ООО «ПРОФКАДРЫ» -лидер кадрового обеспечения крупных  предприятий России.  На крупное предприятие в городе Тихвин требуется Маляр ,  для работы вахтовым методом.  Обязанности:  Работы по нанесению покрытий на металлоконструкции при изготовлении отливок, узлов, вагонов (лакокрасочным и материалами по металлу)  — Требования: — Опыт работы от 1-3 лет. — Наличие удостоверении по специальности и запись в трудовой книжке, подтверждающая квалификацию.  Условия:  — Официальное трудоустройство по ТК РФ. — Оплата отпуска и больничного. — Своевременная зарплата 2 р в месяц на карту любого банка. — Предоставление благоустроенного жилья. — Заселение в день приезда. — Спецодежда и медкомиссия на месте за счёт работодателя. — Компенсация дороги по билетам. — Оплата почасовая 290 р/ч. — График 6/1,по 11 часов, перерывом на обед. — Вахта 60/30,30/30.  Работа в стабильной, динамично развивающейся компании.  Возможность карьерного роста.  Проводим адаптацию и даём полную поддержку новым сотрудникам.",
        "требования к соискателю": "Опыт работы от 1-3 лет. — Наличие удостоверении по специальности и запись в трудовой книжке, подтверждающая квалификацию.",
        "условия": "— Официальное трудоустройство по ТК РФ. — Оплата отпуска и больничного. — Своевременная зарплата 2 р в месяц на карту любого банка. — Предоставление благоустроенного жилья. — Заселение в день приезда. — Спецодежда и медкомиссия на месте за счёт работодателя. — Компенсация дороги по билетам. — Оплата почасовая 290 р/ч. — График 6/1,по 11 часов, перерывом на обед. — Вахта 60/30,30/30.  Работа в стабильной, динамично развивающейся компании.  Возможность карьерного роста.  Проводим адаптацию и даём полную поддержку новым сотрудникам.",
        "описание вакансии": "Вас приветствует компания ООО «ПРОФКАДРЫ» -лидер кадрового обеспечения крупных  предприятий России.  На крупное предприятие в городе Тихвин требуется Маляр ,  для работы вахтовым методом.",
        "обязанности": "Работы по нанесению покрытий на металлоконструкции при изготовлении отливок, узлов, вагонов (лакокрасочным и материалами по металлу)."
    },
    {
        "text": "Крупной Федеральной Компании для работы ВАХТОВЫМ МЕТОДОМ на строительные участки требуются специалисты:  Обязанности:  · Работы на монолите (заливка бетона, укладка и вязка арматуры)  · Монтажные работы (МК/ЖБК)  · Общестроительные работы и вспомогательные работы  Требования:  · Понимание работы в бригаде  · Дисциплинированность, трудолюбие  · Опыт работы в строительстве будет преимуществом  · (готовы рассмотреть кандидатов и без опыта работы, всему научим, в перспективе карьерный рост до бригадира/мастера участка)  Условия:  · Продолжительность вахты 60/30, 90/30 (можно больше)  · Официальное трудоустройство по ТК РФ с первой рабочей смены  · ЗП строго в срок и без задержек (документальное подтверждение в трудовом договоре)  · Авансирование (15 и 30го числа каждого месяца по 15000 р.)  · Обеспечим сезонной спецодеждой и СИЗами без вычетов из ЗП  · Организованное трехраховое горячее питание за счет компании  · Организованные отправки до объектов (покупаем билеты на вахту/с вахты)  · Помощь в прохождении мед. осмотра (при необходимости)  · Возможность получить квалификационное удостоверение (обучение основным и смежным специальностям в нашем аккредитованном учебном центре)",
        "требования к соискателю": "Понимание работы в бригаде  · Дисциплинированность, трудолюбие. Опыт работы в строительстве будет преимуществом  · (готовы рассмотреть кандидатов и без опыта работы, всему научим, в перспективе карьерный рост до бригадира/мастера участка).",
        "условия": "· Продолжительность вахты 60/30, 90/30 (можно больше)  · Официальное трудоустройство по ТК РФ с первой рабочей смены  · ЗП строго в срок и без задержек (документальное подтверждение в трудовом договоре)  · Авансирование (15 и 30го числа каждого месяца по 15000 р.)  · Обеспечим сезонной спецодеждой и СИЗами без вычетов из ЗП  · Организованное трехраховое горячее питание за счет компании  · Организованные отправки до объектов (покупаем билеты на вахту/с вахты)  · Помощь в прохождении мед. осмотра (при необходимости)  · Возможность получить квалификационное удостоверение (обучение основным и смежным специальностям в нашем аккредитованном учебном центре)",
        "описание вакансии": "Крупной Федеральной Компании для работы ВАХТОВЫМ МЕТОДОМ на строительные участки требуются специалисты.",
        "обязанности": "Работы на монолите (заливка бетона, укладка и вязка арматуры)  · Монтажные работы (МК/ЖБК)  · Общестроительные работы и вспомогательные работы."
    }
]

In [12]:
with open('datasetx.json') as f:
    samples = json.load(f)

In [13]:
metrics = {
    "требования к соискателю": [],
    "условия": [],
    "описание вакансии": [],
    "обязанности": []
}

In [14]:
def filter_symbols(text: str):
    t = re.sub(r'[^а-яА-Яa-zA-Z0-9\s]', ' ', text)
    return re.sub(r'\n\t', ' ', t)

In [15]:
from nltk.translate.bleu_score import sentence_bleu
rouge = Rouge()
porter = PorterStemmer()
def get_score(gold, output):
    s = 0
    count = 0
    for k in gold.keys():
        if not gold.get(k, 0) or not output.get(k, 0):
            continue
        gold_point = gold[k]
        gold_point = filter_symbols(gold_point.lower())

        out_point = output[k]
        out_point = filter_symbols(out_point.lower())

        count += 1
        stems = []
        for word in gold_point.split():
            stems.append(porter.stem(word))
        gold_stemmed = ' '.join(stems)
        
        stems = []
        for word in out_point.split():
            stems.append(porter.stem(word))
        output_stemmed = ' '.join(stems)
        print("gg", gold_stemmed, "|", output_stemmed)
        # scores = rouge.get_scores(gold_stemmed, output_stemmed)[0]['rouge-2']['r']
        scores = sentence_bleu(gold_stemmed.split(), output_stemmed.split())
        print(scores)
        s += scores
    return s / count if count != 0 else 0

In [16]:
scores = []

In [17]:
# for sample in tqdm(samples):
#     text = filter_text(sample["исходный текст"].lower())
#     conversation = Conversation()
#     conversation.add_user_message(get_prompt(text))
#     prompt = conversation.get_prompt(tokenizer)

#     output = generate(model, tokenizer, prompt, generation_config)
#     print("out\n", output)
#     print('-' * 15)
#     res = parse_model_answer(output)
#     print("gt:\n", json.dumps({k: v for k, v in sample.items() if k != "исходный текст"}, indent=2, ensure_ascii=False))
#     print("pred\n", json.dumps(res, indent=2, ensure_ascii=False))
#     scores.append(get_score(sample, res))

In [18]:
for sample in tqdm(samples):
    text = filter_text(sample["исходный текст"].lower())
    data = tokenizer([get_prompt(text)], return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items() if k in ("input_ids", "attention_mask")}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)

    print("out\n", output)
    print('-' * 15)
    res = parse_model_answer(output)
    print("gt:\n", json.dumps({k: v for k, v in sample.items() if k != "исходный текст"}, indent=2, ensure_ascii=False))
    print("pred\n", json.dumps(res, indent=2, ensure_ascii=False))
    scores.append(get_score(sample, res))

  0%|          | 0/86 [00:00<?, ?it/s]/home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/matvey/AMOGUS/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it conta

out
 Задание: Дан текст вакансии, выдели из этого текста пункты "название", "специализация", "требования", "условия", "описание", "обязанности" и напиши их в такой форме:
<format-example>
"название": "Бетонщик"
"специализация": "Бетонщики (Вахта,питание, проживание,проезд)"
"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки."
"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет."
"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"
"обязанности": "Мешать бетон стройке"
</format-example>
Если какой-то пункт отсутствует, то оставь после него пустую строку. Пункты кроме "название", "специализация", "требования", "условия", "описание", "обязанности" писать запрещено.
Повторять одинаковые пункты несколько раз запрещено.
Приведу пример вакансии и ответа:
<example>
Текст вакансии: 
<text>
""Срочно требуются бетонщики- монолитчики на работу вахтовым методом!   Tpeбования : Опыт рабoты 

  2%|▏         | 2/86 [00:40<27:31, 19.67s/it]

out
 Задание: Дан текст вакансии, выдели из этого текста пункты "название", "специализация", "требования", "условия", "описание", "обязанности" и напиши их в такой форме:
<format-example>
"название": "Бетонщик"
"специализация": "Бетонщики (Вахта,питание, проживание,проезд)"
"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки."
"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет."
"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"
"обязанности": "Мешать бетон стройке"
</format-example>
Если какой-то пункт отсутствует, то оставь после него пустую строку. Пункты кроме "название", "специализация", "требования", "условия", "описание", "обязанности" писать запрещено.
Повторять одинаковые пункты несколько раз запрещено.
Приведу пример вакансии и ответа:
<example>
Текст вакансии: 
<text>
""Срочно требуются бетонщики- монолитчики на работу вахтовым методом!   Tpeбования : Опыт рабoты 

  3%|▎         | 3/86 [00:50<20:57, 15.16s/it]

...

out
 Задание: Дан текст вакансии, выдели из этого текста пункты "название", "специализация", "требования", "условия", "описание", "обязанности" и напиши их в такой форме:
<format-example>
"название": "Бетонщик"
"специализация": "Бетонщики (Вахта,питание, проживание,проезд)"
"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки."
"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет."
"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"
"обязанности": "Мешать бетон стройке"
</format-example>
Если какой-то пункт отсутствует, то оставь после него пустую строку. Пункты кроме "название", "специализация", "требования", "условия", "описание", "обязанности" писать запрещено.
Повторять одинаковые пункты несколько раз запрещено.
Приведу пример вакансии и ответа:
<example>
Текст вакансии: 
<text>
""Срочно требуются бетонщики- монолитчики на работу вахтовым методом!   Tpeбования : Опыт рабoты 

 99%|█████████▉| 85/86 [27:35<00:32, 32.02s/it]

out
 Задание: Дан текст вакансии, выдели из этого текста пункты "название", "специализация", "требования", "условия", "описание", "обязанности" и напиши их в такой форме:
<format-example>
"название": "Бетонщик"
"специализация": "Бетонщики (Вахта,питание, проживание,проезд)"
"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки."
"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет."
"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"
"обязанности": "Мешать бетон стройке"
</format-example>
Если какой-то пункт отсутствует, то оставь после него пустую строку. Пункты кроме "название", "специализация", "требования", "условия", "описание", "обязанности" писать запрещено.
Повторять одинаковые пункты несколько раз запрещено.
Приведу пример вакансии и ответа:
<example>
Текст вакансии: 
<text>
""Срочно требуются бетонщики- монолитчики на работу вахтовым методом!   Tpeбования : Опыт рабoты 

100%|██████████| 86/86 [28:04<00:00, 19.58s/it]

out
 Задание: Дан текст вакансии, выдели из этого текста пункты "название", "специализация", "требования", "условия", "описание", "обязанности" и напиши их в такой форме:
<format-example>
"название": "Бетонщик"
"специализация": "Бетонщики (Вахта,питание, проживание,проезд)"
"требования": "Опыт рабoты на монолите. Навыки вязки арматуры и установки опалубки."
"условия": "Высокий и стабильный заработок: от 80000 руб. - Комфортные условия проживания за наш счет."
"описание": "Срочно требуются бетонщики- монолитчики на работу вахтовым методом!"
"обязанности": "Мешать бетон стройке"
</format-example>
Если какой-то пункт отсутствует, то оставь после него пустую строку. Пункты кроме "название", "специализация", "требования", "условия", "описание", "обязанности" писать запрещено.
Повторять одинаковые пункты несколько раз запрещено.
Приведу пример вакансии и ответа:
<example>
Текст вакансии: 
<text>
""Срочно требуются бетонщики- монолитчики на работу вахтовым методом!   Tpeбования : Опыт рабoты 

In [19]:
sum(scores[:50]) / len(scores[:50])

3.270820399896461e-232

In [ ]:
len(scores)